In [ ]:
import pandas as pd

from shapely.geometry import Point,Polygon

from shapely.geometry import shape

import numpy as np
import geopandas as gpd

from shapely.geometry import shape, GeometryCollection

from shapely import wkt

import alphashape
import matplotlib.pyplot as plt
from descartes import PolygonPatch
from tqdm import tqdm
from matplotlib import pyplot as plt


def get_gdf(df, geometry, crs):
    try:
        return gpd.GeoDataFrame(df, geometry=geometry, crs='epsg:%s'%str(crs))
    except:
        df[geometry]=df[geometry].apply(lambda x: None if pd.isnull(x)else wkt.loads(x))
        return gpd.GeoDataFrame(df, geometry=geometry, crs='epsg:%s'%str(crs))

def get_points(lat, long):
    """
    As input it accepts 2 Series
    """
    return[ Point (j,i) for i, j in zip(lat, long)]

def get_point_32637(loc):
#     print(loc.x, loc.y)
    inProj = pyproj.CRS("EPSG:4326") # source coordinate system
    outProj = pyproj.CRS("EPSG:32637") # destination coordinate system
    
    return Point(pyproj.transform(inProj,outProj,loc.y,loc.x))

def dist_2fields(g):
    """calculate distance without changing crs.
    As input it accepts Series, usually row of 2 values
    """
    pass
    try:
        return g.iloc[0].distance(g.iloc[1])
    except:
        return 0
    
def area_2fields(g):
    """calculate area without changing crs. 
        As input it accepts Series, usually row of 2 values
    """
    pass
    try:
        return g.iloc[0].intersection(g.iloc[1]).area
    except:
        return 0


def get_h3_net (city_boarders):
    """Create a h3 net for  given city borders.
    borders type : [[]]
    """
    pass
    boarders_gjson= { 
                    "type": "Polygon",
                    "coordinates": [
                       city_boarders 
                    ] 
                }
    hexs = h3.polyfill(boarders_gjson, res=9,geo_json_conformant = True)
    
    polygonise = lambda hex_id: Polygon(\
                            h3.h3_to_geo_boundary(
                            hex_id, geo_json=True)
                                        )
    
    # %time all_polys = gpd.GeoSeries(list(map(polygonise, hexs)), \
    #                                       index=hexs, \
    #                                       crs="EPSG:4326" \
    #                                      )
    
    gdf_hex = gpd.GeoDataFrame(list(map(polygonise, hexs)), \
                                          index=hexs, \
                                          crs="EPSG:4326", \
                     columns=['geometry']\
                                         )
    #     fig, ax=plt.subplots(figsize=(13,10))
    #     gdf_hex.plot(alpha=0.5, linewidth=1, ax=ax)
    #     ctx.add_basemap(ax=ax, crs='epsg:4326')
    #     plt.show()
   
    return gdf_hex

import overpy

def get_osm_objects(api, city, tag, key=None): # query OpenStreetMap data
    if key is None:
        query="""area["name:en"=%s];node[%s](area);out;"""%('"'+city+'"','"'+tag+'"')
    else:
        query="""area["name:en"=%s];node[%s=%s](area);out;"""%('"'+city+'"','"'+tag+'"','"'+key+'"')
    print(query)
    result = api.query(query)
    amenities=set([i.tags[tag] for i in result.nodes])
    return amenities, result
#api = overpy.Overpass()

#######


def make_iso_poly(subgraph ):
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]

    nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
    nodes_gdf = nodes_gdf.set_index('id')
    l=10
    n=0.0065
    while l>4:
        n-=0.0005
        bounding_poly = alphashape.alphashape(nodes_gdf.geometry, n)
        if bounding_poly.geom_type=='Polygon' or bounding_poly.geom_type=='Point':
            l=0
        else:
            l=len(bounding_poly)
        print(n,l)
    return bounding_poly

# n=0
dict_iso=dict()
i=0
# gdf_nodes['iso_'+str(trip_times[n])]=None
for n,r in tqdm(df.reset_index().iterrows()):
#     print(osmid)
    if n>=117:
        mus_node = ox.get_nearest_node(G, (float(r.lat), float(r.long)), method='euclidean')
    #     print(mus_node)
        subgraph = nx.ego_graph(G, mus_node, radius=trip_time, distance='time')
        isochrone_poly = make_iso_poly(subgraph )
        dict_iso[r['index']]=isochrone_poly
#     i+=1
###########

def receive_road_graph(geometry,network_type, crs):
    G = ox.graph_from_polygon(geometry, network_type=network_type)
    # find the centermost node and then project the graph to UTM
    gdf_nodes = ox.graph_to_gdfs(G, edges=True)
    G_pr = ox.project_graph(G,to_crs=crs)
    return G_pr


def  receive_data_from_osm(place_name, dict_names):
#     dict_names = {'amenity': 'kindergarten'}
    tags = dict_names  # insert dict
    # Get the data
    data= ox.geometries_from_place(place_name, tags)




def gdf_join_inter_share(gdf1, gdf2, geocol1, geocol2, merge_col):
    """ receive area of intersections of 2 datasets"""
    gdf_example_pop_open = gpd.sjoin(gdf1,gdf2, op='intersects', how='inner')

    gdf_example_pop_open = gdf_example_pop_open.merge(gdf2[[geocol2,merge_col]], on=merge_col, suffixes=('_x','_y'))

    gdf_example_pop_open['inter_geometry'] = gpd.GeoSeries(gdf_example_pop_open[geocol1+'_x']).intersection(gpd.GeoSeries(gdf_example_pop_open[geocol2+'_y']))

    gdf_example_pop_open['inter_share'] = gpd.GeoSeries(gdf_example_pop_open['inter_geometry']).area/gpd.GeoSeries(gdf_example_pop_open[geocol2+'_y']).area
    #gdf_example_pop_open['pop_weighted'] = gdf_example_pop_open['pop']*gdf_example_pop_open['inter_share'] 
    
    return gdf_example_pop_open


# функции для расчета intersections метрик : betweeness и тд


In [2]:
import geopandas as gpd